When game starts, all states are zero. State system python equivalent = functions for each class. set up class templates in case more than one battle is wanted.
player class
enemy class
animation can be a generic glitch
quantum can be imported later
typical battle:
intro screen where all states are initialized. if start button pressed, hide the screen and initiate runtime
a) roll die to see who goes first b) 

In [116]:
import math
import sys
import os
sys.path.insert(0, os.path.abspath('./..'))
import jupylet.color
from jupylet.sprite import Sprite
from jupylet.label import Label
from jupylet.app import App


In [100]:
app = App()

In [101]:
from qiskit import *
from math import pi

moves are quantum gates
- OBJECTIVE: PLAYER GETS HIGHEST PROBABILITY OF GETTING TO THE 1 STATE
- moves manipulate qubit state
- negative phases are defenses (probably not ?)
- record moves and apply gates to a circuit
- at the end of the game, measure and see who wins
- EXTRA feature: playback: see what moves have been done already
- make room to add more cards
- cards with turn cooldown
Setting up game:
- set up enemy and player moves (are cards that are given prechosen or are they chosen randomly from a predecided array)
- text in top corner showing current vector state of qubit (backend result)
- game start: define cards that the player has
- load up cardUI with # of cards that are in the player array
- function declareWinner

- function measureAndDisplay
    - 1) one by one, send values through a circuit and measure
    - 2) says that it's loading as values run through ionq
    - 3) updates UI probability, vector state, and winning texts
    - 4) returns 1 if player is winning, 0 if player is losing
- function player's move:
    - 1) waits for card user input. if card is on cooldown then deny and repeat this step.
    - 2) return selected value and shrink the selected card to 50
    - 3) puts first value in a list
        ONLY IF PLAY 2 CARDS HAS BEEN SELECTED: 
        - 5) wait for another user response
        - 6) shrink card that was selected to 50
        - 7) puts second value in the list
    - 8) call measuring function
    - 9) call enemyTurn and subtract remaining turns by 1
- function enemy's move:
    - 0) if winning turn number is zero then call declareWinner function and return absolutely nothing
    - 1) #all optimization logic here
    - n-1) call measuring function
    - n) call playerTurn
- how will winning states be declared?
    - 1) if you get |1> you win

In [102]:
# V A R I A B L E S

possibleGatesNames = ("gateName1", "gateName2", "gateName3", "gateName4")
possibleGatesDict = { #FILL THIS OUT (whole game depends on it)
    possibleGatesNames[0] : QuantumCircuit(1).x(0), #replace these with the gate you'll actually be using, add all the gates you want
    possibleGatesNames[1] : QuantumCircuit(1).x(0),
    possibleGatesNames[2] : QuantumCircuit(1).x(0),
    possibleGatesNames[3] : QuantumCircuit(1).x(0)
}

winningTurnNumber = 10 #this can be changed

availableCards = [] #INSERT YOUR GATES *FROM DICTIONARY HERE* (whole game depends on it)
#example: how to set the first card that will be in each hand to the second possible gate 
#availableCards[0] = possibleGatesDict.possibleGatesNames[1]

cardsUI = []
numberOfCards = len(availableCards)

playerHand = availableCards.copy()  #dynamic changing
enemyHand = availableCards.copy()

winning = "Nobody is Currently Winning!"
currentQubitState = "nothing here"
currentQubitProbability = "N/A"



dictionaries example:
#print key with val x
print(list(my_dict.keys())[val_list.index(x)])

In [103]:
# S P R I T E S

playerFace = Sprite('playerDummySprite.png', x=20, y=20, scale=50)
enemyFace = Sprite('enemyDummySprite.png', x=app.width, y=app.height, scale=50)

stateText = Label(currentQubitState, color='cyan', font_size=15, x=64, y=app.height-10, anchor_x='right')
possibleText = Label(currentQubitProbability, color='cyan', font_size=15, x=64, y=app.height-20, anchor_x='right')
winningText = Label(winning, color ='cyan', font_size=18, x=64, y=app.height-30, anchor_x='right')

In [104]:
class Card:
    spr = Sprite('cardDummySprite.png', x=app.width/2, y=20, scale=50)
    def __init__(self, cardType, isShrinking):
        self.cardType = cardType
    
    def showCardType():
        cardText = Label(print(list(possibleGatesDict.keys())[list(possibleGatesDict.values()).index(self.cardType)]), color = 'cyan', font_size=9, x=64, y=app.height-10, anchor_x='right')
        #pass

In [105]:
# state = State ()

In [106]:
def cardShrink(card):
    while card.scale > 50 and card.isShrinking is True:
        card.scale -= 1

In [107]:
for i in range(numberOfCards):
    cardsUI[i] = Card(playerHand[i], False)
    cardsUI[i].spr.x += (i*(scale)) + 4

In [108]:
@app.event
def render(ct,dt):
    app.window.clear()
    playerFace.draw()
    enemyFace.draw()
    for i in range (numberOfCards):
        cardsUI[i].draw()

In [109]:
def measureAndDisplay():
    """
    - 1) one by one, send values through a circuit and measure
    - 2) says that it's loading as values run through ionq
    - 3) updates UI probability, vector state, and winning texts
    - 4) returns 1 if player is winning, 0 if player is losing
    """
    #to change text: declaredLable.text = str(intValue)

In [110]:
def declareWinner():
    if measureAndDisplay() == 1:
        winningText.text = "YOU WIN!"
    elif measureAndDisplay() == 0:
        winningText.text = "You lost. Go try again!"

In [111]:
def playerTurn():
    """
    
    """

In [112]:
def enemyTurn():
    pass

In [113]:
def updateCards():
    pass

In [114]:
def updateUI():
    pass
    

In [115]:
app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…